# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

GOOGLE_API_KEY=****4cz4
LANGSMITH_API_KEY=****8c39
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain.agents import create_agent

In [ ]:
agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [11]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Okay, here's one:

Why did the full-stack developer break up with the frontend developer?

...Because he wanted to see other frameworks!

(And maybe even build a database for his feelings.)


## values
You have seen this streaming mode in our examples so far. 

In [12]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why don't scientists trust atoms?

Because they make up everything!


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [13]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

In a world of wonder, big and small,
A little giggle answers every call.
The sunbeams dance upon the floor,
And dreams come knocking at your door.

A fluffy cloud, a robin's song,
Where happy moments all belong.
A painted rainbow, bright and grand,
A gentle breeze across the land.

So let's explore with curious eyes,
Beneath the wide and sunny skies.
With kindness shared and spirits bright,
We fill our days with pure delight!

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [14]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='ea5b62b3-b8e0-459a-a838-74ca283ca3e2')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='ea5b62b3-b8e0-459a-a838-74ca283ca3e2'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "SF"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--2e9f23ca-9d26-41b5-877f-b2cf492ef0f7-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': '1cd3e666-58fb-4175-b0f9-d9492af855f5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 47, 'output_tokens': 15, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}})]})
('custom', 'Looking up data for city: SF')
('cus

In [15]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: SF')
('custom', 'Acquired data for city: SF')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [16]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: SF
Acquired data for city: SF
